## Code for simulating the SDE model

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import random
import ewstools
from ewstools.models import simulate_ricker

In [ ]:
def param_init():
    # initiate parameters
    R0 = np.random.uniform(12, 18)
    gamma = np.random.uniform(365/22, 365/13)
    mu = np.random.uniform(1/70, 1/50)
    kappa = 3000
    delta = 3e-4
    N = 200000
    N1 = N*np.random.uniform(0.05, 0.4)
    h = np.random.uniform(0.4, 0.8)
    c = np.random.uniform(10, 200)
    tau = np.random.uniform(50000, 100000)
    post_prob = 0.2
    param = {'R0':R0, 'gamma':gamma, 'mu':mu, 'kappa':kappa, 'delta':delta, 'N':N, 'N1':N1, 'h':h, 'c':c,
             'tau':tau, 'post_prob':post_prob}
    return param

In [ ]:
def valid_plot(I, threshold):
    # A plot is valid if the infected proportion is no larger than the threshold
    # Abnormal realizations with large infected proportion will be disgarded
    return max(I) <= threshold

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def transition(x, thres):
    # Return the transition time point, or -1 if no transition occurs
    for i in range(len(x)):
        if x[i] < thres:
            return i
    return -1

In [ ]:
def levy_sde_media(param, ic, sigma, tf, dt):
    # param: dict of parameters
    # ic: list of initial conditions
    # sigma: list of noise amplitudes
    # tf: final time
    # dt: timestep
    
    R0 = param['R0']
    gamma = param['gamma']
    mu = param['mu']
    beta = R0*(gamma+mu)
    kappa = param['kappa']
    delta = param['delta']
    N = param['N']
    N1 = param['N1']
    h = param['h']
    c = param['c']
    tau = param['tau']
    
    s, i, x1, x2 = ic[0], ic[1], ic[2], ic[3]
    N2 = N - N1
    x = (x1*N1 + x2*N2)/N
    tw_orig = c*x1*(1-x1) + tau*i
    tw = tw_orig
    
    S, I, X1, X2, X, Tw = [s], [i], [x1], [x2], [x], [tw]
    
    tot = int(tf/dt)
    alpha = 1.5
    while True:  # get rid of extremely large noises
        r = sp.stats.levy_stable.rvs(alpha=alpha, beta=0, size=tot)
        if max(r) < 200 and min(r) > -200:
            break

    t = 0
    j = 1
    T = [t]
    while j <= tot:
        t += dt

        # increasing vaccine risk
        omega = (t*0.5 + 2)*1e-4

        A1 = -omega + i + delta*(2*x1-1 + (1-h)*(2*x2-1))
        A2 = -omega + i + delta*(2*x2-1 + (1-h)*(2*x1-1))
        A3 = -omega + i + delta*(2-h)*(x1+x2-1)

        f_s = mu*(1-x) - beta*np.exp(-0.6*i)*s*i - mu*s
        f_i = beta*np.exp(-0.6*i)*s*i - (gamma+mu)*i
        f_x1 = kappa*x1*(1-x1)*A1 + (1-h)*kappa*(x2*(1-x1)*max(A3,0) - (1-x2)*x1*max(-A3,0))
        f_x2 = kappa*x2*(1-x2)*A2 + (1-h)*kappa*(x1*(1-x2)*max(A3,0) - (1-x1)*x2*max(-A3,0))

        dL = r[j-1]

        s = min(max(s + f_s*dt + (dt**(1/alpha))*sigma[0]*dL, 0), 1)
        i = min(max(i + f_i*dt + (dt**(1/alpha))*sigma[1]*dL, 0), 1)
        x1 = min(max(x1 + f_x1*dt + (dt**(1/alpha))*sigma[2]*dL, 0), 1)
        x2 = min(max(x2 + f_x2*dt + (dt**(1/alpha))*sigma[3]*dL, 0), 1)
        
        # N1 and N2 are changing over time
        # growth rate of N1 is approx 5% per year
        N1 = N1*(1 + 0.05/1000)
        N2 = N - N1
        x = (x1*N1 + x2*N2)/N

        # baseline Tw number
        tw_orig = c*x1*(1-x1) + tau*i
        
        prob = np.random.uniform()
        post_prob = 0.2+0.8*sigmoid(Tw[-1]-100)
        
        k = 0.2
        while True:
            rt = sp.stats.levy_stable.rvs(alpha=1.2, beta=0.25-1/(tw_orig+4), scale=1+0.2*sigmoid((tw_orig-50)/20), loc=(1-k)*tw_orig)
            if rt < min(2*c, 5*tw_orig):
                break
        tw = max(k*tw_orig+rt,0)
        if prob >= post_prob:
            tw = sum(np.random.binomial(1, 0.1, int(tw)))

        T.append(t)
        S.append(s)
        I.append(i)
        X1.append(x1)
        X2.append(x2)
        X.append(x)
        Tw.append(tw)
        
        j += 1

    return T, S, I, X1, X2, X, Tw

In [ ]:
def levy_sde_media_null(param, ic, sigma, tf, dt):
    # ic: list of initial conditions
    # tf: final time
    # dt: timestep
    
    R0 = param['R0']
    gamma = param['gamma']
    mu = param['mu']
    beta = R0*(gamma+mu)
    kappa = param['kappa']
    delta = param['delta']
    N = param['N']
    N1 = param['N1']
    h = param['h']
    c = param['c']
    tau = param['tau']
    
    s, i, x1, x2 = ic[0], ic[1], ic[2], ic[3]
    N2 = N - N1
    x = (x1*N1 + x2*N2)/N
    tw_orig = c*x1*(1-x1) + tau*i
    tw = tw_orig
    
    S, I, X1, X2, X, Tw = [s], [i], [x1], [x2], [x], [tw]
    
    tot = int(tf/dt)
    alpha = 1.5
    while True:  # get rid of extremely large noises
        r = sp.stats.levy_stable.rvs(alpha=alpha, beta=0, size=tot)
        if max(r) < 200 and min(r) > -200:
            break

    t = 0
    j = 1
    T = [t]
    while j <= tot:
        t += dt

        # constant vaccine risk
        omega = np.random.uniform(0.7, 0.9)*delta*(2-h)

        A1 = -omega + i + delta*(2*x1-1 + (1-h)*(2*x2-1))
        A2 = -omega + i + delta*(2*x2-1 + (1-h)*(2*x1-1))
        A3 = -omega + i + delta*(2-h)*(x1+x2-1)

        f_s = mu*(1-x) - beta*np.exp(-0.6*i)*s*i - mu*s
        f_i = beta*np.exp(-0.6*i)*s*i - (gamma+mu)*i
        f_x1 = kappa*x1*(1-x1)*A1 + (1-h)*kappa*(x2*(1-x1)*max(A3,0) - (1-x2)*x1*max(-A3,0))
        f_x2 = kappa*x2*(1-x2)*A2 + (1-h)*kappa*(x1*(1-x2)*max(A3,0) - (1-x1)*x2*max(-A3,0))

        dL = r[j-1]

        s = min(max(s + f_s*dt + (dt**(1/alpha))*sigma[0]*dL, 0), 1)
        i = min(max(i + f_i*dt + (dt**(1/alpha))*sigma[1]*dL, 0), 1)
        x1 = min(max(x1 + f_x1*dt + (dt**(1/alpha))*sigma[2]*dL, 0), 1)
        x2 = min(max(x2 + f_x2*dt + (dt**(1/alpha))*sigma[3]*dL, 0), 1)
        
        # N1 and N2 are changing over time
        # growth rate of N1 is approx 5% per year
        N1 = N1*(1 + 0.05/1000)
        N2 = N - N1
        x = (x1*N1 + x2*N2)/N

        tw_orig = c*x1*(1-x1) + tau*i
        
        prob = np.random.uniform()
        post_prob = 0.2+0.8*sigmoid(Tw[-1]-80)
        
        k = 0.2
        while True:
            rt = sp.stats.levy_stable.rvs(alpha=1.2, beta=0.25-1/(tw_orig+4), scale=1+0.2*sigmoid((tw_orig-50)/20), loc=(1-k)*tw_orig)
            if rt < min(2*c, 5*tw_orig):
                break
        tw = max(k*tw_orig+rt,0)
        if prob >= post_prob:
            tw = sum(np.random.binomial(1, 0.1, int(tw)))

        T.append(t)
        S.append(s)
        I.append(i)
        X1.append(x1)
        X2.append(x2)
        X.append(x)
        Tw.append(tw)
        
        j += 1

    return T, S, I, X1, X2, X, Tw

In [ ]:
# Positive samples
# For the convenience of demonstration, we run a relatively small number of simulations
# The training set contains 10000 positive samples
num_simu_1 = 10

In [ ]:
df_s = pd.DataFrame()
df_i = pd.DataFrame()
df_x1 = pd.DataFrame()
df_x2 = pd.DataFrame()
df_x = pd.DataFrame()
df_tw = pd.DataFrame()
thres_list = []
trans_list = []

In [ ]:
i = 0
while i < num_simu_1:
    check = False
    param = param_init()
    while not(check):
        T, S, I, X1, X2, X, Tw = levy_sde_media(param, [0.01, 1e-6, 0.999, 0.999], [1e-3, 2e-6, 4e-3, 1e-3], 12, 1e-3)
        check = valid_plot(I, 1e-2)  # get rid of very abnormal scenarios
    df = pd.DataFrame.from_dict({'time':T, 'S':S, 'I':I, 'X1':X1, 'X2':X2, 'X':X, 'Tw':Tw})
    
    thres = 1-20*(4e-3*param['N1'] + 1e-3*(param['N']-param['N1']))/param['N']
    trans = transition(df['X'].groupby(np.arange(len(df['I'])) // 3).mean(), thres)
    if trans == -1:  # ignore cases when transition has not occurred
        continue
    else:
        thres_list.append(thres)
        trans_list.append(trans)
    
        # interpolation for s, i, x1, x2, x
        df_s[i] = df['S'].groupby(np.arange(len(df['S'])) // 3).mean()
        df_i[i] = df['I'].groupby(np.arange(len(df['I'])) // 3).mean()
        df_x1[i] = df['X1'].groupby(np.arange(len(df['X1'])) // 3).mean()
        df_x2[i] = df['X2'].groupby(np.arange(len(df['X2'])) // 3).mean()
        df_x[i] = df['X'].groupby(np.arange(len(df['X'])) // 3).mean()
    
        # aggregate for tw
        df_tw[i] = df['Tw'].groupby(np.arange(len(df['Tw'])) // 3).sum()
        i += 1

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=3, figsize=(15,6))
for i, ax in zip(range(6), ax1.flatten()):
    ax.plot(df_x.index, df_x[i], label=r'$X$', linewidth=2, color=plt.cm.Blues(128))
    ax.set_ylim(bottom=0.95, top=1.0)
    ax.axhline(y=thres_list[i], linestyle='--', color='k', linewidth=1)
fig1.tight_layout()

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=3, figsize=(15,6))
for i, ax in zip(range(6), ax1.flatten()):
    ax.plot(df_tw.index, df_tw[i], label=r'$Tw$', linewidth=2, color=plt.cm.Purples(128))
    ax.set_ylim(bottom=-1, top=50)
    ax.set_xlim(left=trans_list[i]-600, right=trans_list[i]+100)
    ax.axvline(x=trans_list[i]-500, linestyle='--', color='k', linewidth=1)
    ax.axvline(x=trans_list[i], linestyle='--', color='k', linewidth=1)
fig1.tight_layout()

In [ ]:
# Neutral samples
# For the convenience of demonstration, we run a relatively small number of simulations
# The training set contains 10000 neutral samples
num_simu_0 = 10

In [ ]:
i = 0
while i < num_simu_0:
    check = False
    param = param_init()
    while not(check):
        T, S, I, X1, X2, X, Tw = levy_sde_media_null(param, [0.01, 1e-6, 0.999, 0.999], [1e-3, 2e-6, 4e-3, 1e-3], 12, 1e-3)
        check = valid_plot(I, 1e-2)  # get rid of very abnormal scenarios
    df = pd.DataFrame.from_dict({'time':T, 'S':S, 'I':I, 'X1':X1, 'X2':X2, 'X':X, 'Tw':Tw})
    
    thres = 1-20*(4e-3*param['N1'] + 1e-3*(param['N']-param['N1']))/param['N']
    trans = transition(df['X'].groupby(np.arange(len(df['I'])) // 3).mean(), thres)
    if trans != -1:  # ignore cases when transition has occurred
        continue
    else:
        thres_list.append(thres)
        trans_list.append(trans)
    
        # interpolation for s, i, x1, x2, x
        j = num_simu_1 + i
        df_s[j] = df['S'].groupby(np.arange(len(df['S'])) // 3).mean()
        df_i[j] = df['I'].groupby(np.arange(len(df['I'])) // 3).mean()
        df_x1[j] = df['X1'].groupby(np.arange(len(df['X1'])) // 3).mean()
        df_x2[j] = df['X2'].groupby(np.arange(len(df['X2'])) // 3).mean()
        df_x[j] = df['X'].groupby(np.arange(len(df['X'])) // 3).mean()
    
        # aggregate for tw
        df_tw[j] = df['Tw'].groupby(np.arange(len(df['Tw'])) // 3).sum()
        i += 1

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=3, figsize=(15,6))
for i, ax in zip(range(6), ax1.flatten()):
    j = num_simu_1 + i
    ax.plot(df_x.index, df_x[j], label=r'$X$', linewidth=2, color=plt.cm.Blues(128))
    ax.set_ylim(bottom=0.95, top=1.0)
    ax.axhline(y=thres_list[j], linestyle='--', color='k', linewidth=1)
fig1.tight_layout()

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=3, figsize=(15,6))
for i, ax in zip(range(6), ax1.flatten()):
    j = num_simu_1 + i
    ax.plot(df_tw.index, df_tw[j], label=r'$Tw$', linewidth=2, color=plt.cm.Purples(128))
    ax.set_ylim(bottom=-1, top=50)
    ax.set_xlim(left=3500, right=4000)
fig1.tight_layout()

In [ ]:
df_pt = pd.DataFrame({'thres':thres_list, 'trans':trans_list})

## Code for generating training samples

In [ ]:
# Variables used from previous section: df_tw, df_pt

In [ ]:
df_ts = pd.DataFrame()
df_sm = pd.DataFrame()
df_res = pd.DataFrame()
lb_list = []

In [ ]:
ts_length = 500

In [ ]:
for i in range(df_tw.shape[1]):
    trans = df_pt['trans'][i]
    
    if trans != -1:
        lb_list.append(1)
        ts = ewstools.TimeSeries(data=df_tw[i], transition=trans)  # <=trans
        
        # Detrend
        ts.detrend(method='Gaussian', bandwidth=100)  # 100 data points
        df_ts[i] = ts.state[trans-ts_length:trans].reset_index()['state']  # ts_length data points before trans
        df_sm[i] = ts.state[trans-ts_length:trans].reset_index()['smoothing']
        res = ts.state[trans-ts_length:trans].reset_index()['residuals']
        df_res[i] = res/np.mean(np.abs(res))  # l1-normalized
    else:
        lb_list.append(0)
        ts = ewstools.TimeSeries(data=df_tw[i])  # entire time series
        
        # Detrend
        ts.detrend(method='Gaussian', bandwidth=100)  # 100 data points
        df_ts[i] = ts.state.dropna()[-ts_length:].reset_index()['state']  # last ts_length data points
        df_sm[i] = ts.state.dropna()[-ts_length:].reset_index()['smoothing']
        res = ts.state.dropna()[-ts_length:].reset_index()['residuals']
        df_res[i] = res/np.mean(np.abs(res))  # l1-normalized

df_lb = pd.DataFrame({'label':lb_list})

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=3, figsize=(15,6))
for i, ax in zip(range(6), ax1.flatten()):
    ax.plot(df_ts.index, df_ts[i], linewidth=2, color=plt.cm.Purples(64))
    ax.plot(df_ts.index, df_sm[i], linewidth=2, color=plt.cm.Purples(192))
    ax.set_ylim(bottom=0, top=50)
fig1.tight_layout()

In [ ]:
fig1, ax1 = plt.subplots(nrows=2, ncols=3, figsize=(15,6))
for i, ax in zip(range(6), ax1.flatten()):
    j = num_simu_1 + i
    ax.plot(df_ts.index, df_ts[j], linewidth=2, color=plt.cm.Purples(64))
    ax.plot(df_ts.index, df_sm[j], linewidth=2, color=plt.cm.Purples(192))
    ax.set_ylim(bottom=0, top=50)
fig1.tight_layout()

In [ ]:
# Save file
#df_ts.to_csv('./train_data/train_ts.csv', index=None)
#df_sm.to_csv('./train_data/train_sm.csv', index=None)
#df_res.to_csv('./train_data/train_res.csv', index=None)
#df_lb.to_csv('./train_data/train_lb.csv', index=None)
#df_pt.to_csv('./train_data/train_pt.csv', index=None)

For generating testing samples, modify noise amplitudes (sigma) and repeat the above steps, or simply use part of the training set that has not been used for training

## Code for generating testing metrics

Generating variance and lag-1 autocorrelation for $x$ and $T_p$ and Kendall tau for plotting ROC curves

In [ ]:
# Variables used from previous section: df_x, df_tw, df_pt

In [ ]:
df_x_ts = {}
df_x_sm = {}
df_x_var = {}
df_x_ac = {}

In [ ]:
df_tw_ts = {}
df_tw_sm = {}
df_tw_var = {}
df_tw_ac = {}

In [ ]:
ktau_x_dict = {'var':[], 'ac':[]}
ktau_tw_dict = {'var':[], 'ac':[]}

In [ ]:
for i in range(df_x.shape[1]):  # the same as df_tw.shape[1]
    trans = df_pt['trans'][i]
    
    if trans != -1:
        x_ts = ewstools.TimeSeries(data=df_x[i], transition=trans)
        tw_ts = ewstools.TimeSeries(data=df_tw[i], transition=trans)
    else:
        trans = df_x.shape[0]
        x_ts = ewstools.TimeSeries(data=df_x[i])  # entire time series
        tw_ts = ewstools.TimeSeries(data=df_tw[i])  # entire time series
    
    # Detrend
    x_ts.detrend(method='Gaussian', bandwidth=400)
    df_x_ts[i] = x_ts.state['state']
    df_x_sm[i] = x_ts.state['smoothing']
    tw_ts.detrend(method='Gaussian', bandwidth=100)
    df_tw_ts[i] = tw_ts.state['state']
    df_tw_sm[i] = tw_ts.state['smoothing']
    
    # EWS
    x_ts.compute_var(rolling_window=200)
    x_ts.compute_auto(rolling_window=200, lag=1)
    df_x_var[i] = x_ts.ews['variance']
    df_x_ac[i] = x_ts.ews['ac1']
    
    tw_ts.compute_var(rolling_window=200)
    tw_ts.compute_auto(rolling_window=200, lag=1)
    df_tw_var[i] = tw_ts.ews['variance']
    df_tw_ac[i] = tw_ts.ews['ac1']
    
    # Kendall tau
    x_ts.compute_ktau(tmin=0, tmax=trans)  # tmax=trans/trans-100/trans-200/trans-400
    ktau_x_dict['var'].append(x_ts.ktau['variance'])
    ktau_x_dict['ac'].append(x_ts.ktau['ac1'])
    
    tw_ts.compute_ktau(tmin=0, tmax=trans)  # tmax=trans/trans-100/trans-200/trans-400
    ktau_tw_dict['var'].append(tw_ts.ktau['variance'])
    ktau_tw_dict['ac'].append(tw_ts.ktau['ac1'])

In [ ]:
df_ktau_x = pd.DataFrame(ktau_x_dict)
df_ktau_tw = pd.DataFrame(ktau_tw_dict)

In [ ]:
# Save files
#df_x_var.to_csv('./test_data/test_x_var.csv', index=None)
#df_x_ac.to_csv('./test_data/test_x_ac.csv', index=None)
#df_tw_var.to_csv('./test_data/test_tw_var.csv', index=None)
#df_tw_ac.to_csv('./test_data/test_tw_ac.csv', index=None)
#df_ktau_x.to_csv('./test_data/test_ktau_x.csv', index=None)
#df_ktau_tw.to_csv('./test_data/test_ktau_tw.csv', index=None)